In [1]:
import numpy as np
import pandas as pd
import re
import torch
from tqdm import notebook
from sklearn.model_selection import train_test_split, GridSearchCV
from pymystem3 import Mystem
m = Mystem()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import torch
import transformers as ppb
import lightgbm as lgb
from pytorch_transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import wordnet
import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import word_tokenize
import requests
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from nltk.probability import FreqDist
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

from lightgbm import LGBMClassifier




INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
data = pd.read_excel('text.xlsx')

In [5]:
data

,category,text
0,1,Капитальный ремонт нежилых помещений (склада)
1,1,Капитальный ремонт нежилых помещений (склада)
2,1,Капитальный ремонт нежилых помещений (склада)
3,1,Капитальный ремонт нежилых помещений (склада)
4,1,Капитальный ремонт нежилых помещений (склада)
...,...,...
8842,2,Капитальный ремонт гидроизоляции фундаментов и...
8843,2,Капитальный ремонт гидроизоляции фундаментов и...
8844,2,Капитальный ремонт гидроизоляции фундаментов и...
8845,2,Капитальный ремонт гидроизоляции фундаментов и...


In [8]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from nltk.corpus import stopwords as nltk_stopwords
import pandas as pd
import joblib

# Загрузите данные

nltk_stopwords = list(nltk_stopwords.words('russian'))
extra_stopwords = [
    # (Ваш список стоп-слов)
]
nltk_stopwords.extend(extra_stopwords)
count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

def train_model(target_col):
    
    # Извлекаем признаки и целевую переменную
    X = data['text']
    y = data[target_col].values

    # Разделите данные на тренировочные и тестовые наборы

    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    except ValueError:
        # В случае ошибки отключаем стратификацию
        print(f"Stratification failed for {target_col}. Proceeding without stratification.")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Примените TF-IDF векторизатор к вашим данным
    tfidf_train = count_tf_idf.fit_transform(X_train)
    tfidf_test = count_tf_idf.transform(X_test)

    # Определите конвейер и сетку параметров для поиска по сетке
    pipe = Pipeline([
        ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
    ])

    param_grid = [
        {
            'model': [LogisticRegression(random_state=42, solver='liblinear')],
            'model__C': list(range(1, 15, 3)),
            'model__penalty': ['l1', 'l2']
        },
        {
            'model': [SVC(random_state=42)],
            'model__C': [0.1, 1, 10],
            'model__kernel': ['linear', 'rbf']
        },
    ]

    # Проведите поиск по сетке, чтобы найти наилучшие параметры
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(f1_score, average='weighted'), cv=5, verbose=True, n_jobs=-1)
    best_grid = grid.fit(tfidf_train, y_train)
    
    # Сохраните модель
    joblib.dump(best_grid, f"{target_col}_model.joblib")

    # Выведите наилучшие параметры и оценку
    print(f"Best parameters for {target_col} are:", grid.best_params_)
    print(f"Best cross-validation score for {target_col} is:", grid.best_score_)

    # Проверка на тестовой выборке
    test_predictions = best_grid.predict(tfidf_test)
    test_score = f1_score(y_test, test_predictions, average='weighted')
    print(f"Test F1 score for {target_col} is:", test_score)
    
    return grid.best_score_, test_score

# Создание словаря для хранения обученных моделей
models = {}

# Обучите модели для каждого агентства
 
models = train_model('category')



Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for category are: {'model': LogisticRegression(C=1, random_state=42, solver='liblinear'), 'model__C': 1, 'model__penalty': 'l2'}
Best cross-validation score for category is: 1.0
Test F1 score for category is: 1.0


In [14]:
data = pd.read_excel('text.xlsx')

In [15]:
import pandas as pd

data = data.drop_duplicates(subset='text', keep='first')


In [16]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Хотите BERT вместо distilBERT? Раскомментируйте следующую строку:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
tokenized = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=200,)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [18]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [19]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.tensor(np.array(padded[batch_size*i:batch_size*(i+1)])) 
        attention_mask = torch.tensor(np.where(batch != 0, 1, 0))
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
        np.save('example_1', embeddings)

0it [00:00, ?it/s]

In [22]:
features = np.concatenate(np.load('example_1.npy'))
target = data['category']
train_features, test_features, train_target, test_target = train_test_split(features, target)
train_features, features_vr, train_target, target_vr = train_test_split(features, target, test_size = 0.4, random_state = 42)
valid_features, test_features, valid_target, test_target = train_test_split(features_vr, target_vr, test_size=0.5, random_state = 42)

ValueError: Found input variables with inconsistent numbers of samples: [100, 29]

In [ ]:
modeling(LogisticRegression(random_state=12345, C = 4, penalty = 'l1', solver='liblinear', max_iter=200),
         train_features, train_target, valid_features, valid_target) 
modeling(lgb.LGBMClassifier(), train_features, train_target, valid_features, valid_target)